In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Example accessing a CSV file
train_data_path = '/content/drive/MyDrive/Colab Notebooks/1_classification/Train/'
test_path = '/content/drive/MyDrive/Colab Notebooks/1_classification/Test/'

**DATA-LOAD**

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


# Create generators for test and ground truth data
train_dataset = image_dataset_from_directory(
    train_data_path,
    batch_size=16,          # Batch size for training
    image_size=(512, 512),  # Target image size              # Random seed for shuffling
)

# Create a validation dataset
validation_dataset = image_dataset_from_directory(
    test_path,
    batch_size=16,
    image_size=(512, 512),
    shuffle=False  # No need to shuffle validation data
)
class_names = validation_dataset.class_names


Found 800 files belonging to 2 classes.
Found 200 files belonging to 2 classes.


**MODEL**

In [5]:
import tensorflow as tf


model  = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu', input_shape=(512, 512, 3)))
model.add(tf.keras.layers.MaxPooling2D(2, 2))
model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2))
# model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(2, 2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))


model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 253, 253, 32)      18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 508032)            0         
                                                                 
 dense (Dense)               (None, 1)                 5

In [6]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),

              metrics=['accuracy'])

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='loss',  # Metric to monitor
                               patience=5,         # Number of epochs with no improvement after which training will be stopped
                               restore_best_weights=True)  # Restore model weights from the epoch with the best value of the monitored metric


In [17]:
history = model.fit(
          train_dataset,

      steps_per_epoch=4,
      epochs=50,
      verbose=1,
          callbacks=[early_stopping])

Epoch 1/50
4/4 [==============================] - 58s 12s/step - loss: 0.2405 - accuracy: 0.9375
Epoch 2/50
4/4 [==============================] - 43s 11s/step - loss: 0.0930 - accuracy: 0.9844
Epoch 3/50
4/4 [==============================] - 46s 12s/step - loss: 0.0318 - accuracy: 1.0000
Epoch 4/50
4/4 [==============================] - 43s 10s/step - loss: 0.3282 - accuracy: 0.8750
Epoch 5/50
4/4 [==============================] - 41s 10s/step - loss: 11.7837 - accuracy: 0.6094
Epoch 6/50
4/4 [==============================] - 41s 11s/step - loss: 1.3713 - accuracy: 0.8125
Epoch 7/50
4/4 [==============================] - 42s 10s/step - loss: 0.3687 - accuracy: 0.8750
Epoch 8/50
4/4 [==============================] - 42s 10s/step - loss: 0.2501 - accuracy: 0.9219


In [19]:
model.save('/content/drive/MyDrive/Colab Notebooks/id_card_classifier2.h5')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/Colab Notebooks/id_card_classifier.h5')


In [18]:
predictions = model.predict(validation_dataset)


# Get the class predictions (0 or 1 for binary classification)
class_predictions = (predictions > 0.5).astype(int)
validation_result = model.evaluate(validation_dataset)

# Displaying the class predictions
print("Class Predictions:", (class_predictions.sum()))


13/13 [==============================] - 43s 3s/step - loss: 0.2858 - accuracy: 0.9250
Class Predictions: 97


# New Section

In [1]:
for images, labels in validation_dataset:
    # Predictions
    predictions = model.predict(images)
    class_predictions = (predictions > 0.5).astype(int)

    # Iterate over the batch
    for i in range(len(images)):
        # Print the actual image and its prediction
        print("Prediction:", class_predictions[i])
        print("Actual Image:")
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.show()

NameError: name 'validation_dataset' is not defined

In [ ]:
print("test Accuracy:", validation_result[1])

test Accuracy: 0.9350000023841858


# New Section